In [1]:
import numpy as np
import pandas as pd
from pandas import DataFrame
import scipy as sp
import string
import json
import re
import sklearn.metrics as metrics
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as  plt
import seaborn as sns
import sklearn
from sklearn.metrics import confusion_matrix
####
from sklearn.model_selection import train_test_split, cross_val_predict
from sklearn import linear_model
import statsmodels.api as sm

from sklearn.linear_model import LogisticRegression
import sklearn.linear_model as linear_model
import sklearn.model_selection as model_selection

from sklearn import svm
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB

from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import KMeans

In [2]:
# open training dataset and parse recipes

with open('train.json') as cooking_file:  
    
    data = json.load(cooking_file)
    

In [3]:
#COUNTVECTORIZER
#removing punctuations and spaces before fixing the data and keep them all in a dict  (italian, mexican)

punctuations = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''

def gather_recipes(recipes, cuisine_country):
    
    data_list = []
    
    for _object in recipes:
        
        if _object.get('cuisine') == cuisine_country:
            
            ingredients = _object.get('ingredients') 

            _id = _object.get('id')
            
            for i in ingredients:
                
                _dict = dict()
                ingr = i.replace(' ', '').lower()
                    
                for ch in ingr: 
                    
                    if ch in punctuations: 
                        
                        ingr = ingr.replace(ch, '')  
                
                _dict['ingredients'] = ingr                
                _dict['id'] = _id
                data_list.append(_dict)
                
    return data_list


clean_italian_cuisine_data = gather_recipes(data, 'italian')
clean_mexican_cuisine_data = gather_recipes(data, 'mexican')

clean_filipino_cuisine_data = gather_recipies(data, 'filipino')
clean_british_cuisine_data = gather_recipies(data, 'british')
clean_moroccan_cuisine_data = gather_recipies(data, 'moroccan')

In [4]:
#creating dfs  (italian, mexican)

italian_df = DataFrame(clean_italian_cuisine_data)
italian_ingredients = list(set(italian_df.ingredients))

mexican_df = DataFrame(clean_mexican_cuisine_data)
mexican_ingredients = list(set(mexican_df.ingredients))


#creating dfs (filipino, british, moroccan )

filipino_df = DataFrame(clean_filipino_cuisine_data) 
filipino_ingredients = list(set(filipino_df.ingredients))

british_df = DataFrame(clean_british_cuisine_data)
british_ingredients = list(set(british_df.ingredients))

moroccan_df = DataFrame(clean_moroccan_cuisine_data)
moroccan_ingredients = list(set(moroccan_df.ingredients))

In [5]:
#fixing data for CountVectorizer (italian, mexican)

it_unique_ids = []
it_ingredients = []
it_grouped = italian_df.groupby('id')

for ids, ing in it_grouped:
    
    it_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    it_ingredients.append(row)

    
mex_unique_ids = []
mex_ingredients = []
mex_grouped = mexican_df.groupby('id')

for ids, ing in mex_grouped:
    
    mex_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    
    mex_ingredients.append(row)

In [6]:
#using CountVectorizer  (italian, mexican) and adding one column Cuisine

it_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
it_array = it_vectorizer.fit_transform(it_ingredients)
it_array = it_array.toarray()
it_df = DataFrame(it_array, columns = it_vectorizer.get_feature_names(), index = it_unique_ids)
it_df['Cuisine'] = 0 #italian
it_df.to_csv('it_cleandata.csv', index_label = 'ID')
#print(it_df)

mex_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
mex_array = mex_vectorizer.fit_transform(mex_ingredients)
mex_array = mex_array.toarray()
mex_df = DataFrame(mex_array, columns = mex_vectorizer.get_feature_names(), index = mex_unique_ids)
mex_df['Cuisine'] = 1 #mexican
mex_df.to_csv('mex_cleandata.csv', index_label = 'ID')
#print(mex_df)

In [7]:
#putting mexican and italian in one DataFrame

df = pd.DataFrame(it_df)
df = df.append(mex_df, sort = False)
df.fillna(0, inplace = True)
df = df.astype('int64')

In [8]:
#TFIDFVECTORIZER
#removing punctuations before fixing the data and keep them all in a dict  (italian, mexican)

def gather_recipes(recipes, cuisine_country):
    
    data_list = []
    
    for _object in recipes:
        
        if _object.get('cuisine') == cuisine_country:
            
            ingredients = _object.get('ingredients') 
            
            _id = _object.get('id')
            
            for i in ingredients:
                
                _dict = dict()
                ingr = i.lower()
                    
                for ch in ingr: 
                    
                    if ch in punctuations: 
                        
                        ingr = ingr.replace(ch, '')  
                
                _dict['ingredients'] = ingr                
                _dict['id'] = _id
                data_list.append(_dict)
                
    return data_list

clean2_italian_cuisine_data = gather_recipes(data, 'italian')
clean2_mexican_cuisine_data = gather_recipes(data, 'mexican')

In [9]:
#creating dfs2  (italian, mexican)

italian2_df = DataFrame(clean2_italian_cuisine_data)
mexican2_df = DataFrame(clean2_mexican_cuisine_data)

In [10]:
#fixing data for TfidfVectorizer (italian, mexican)

it2_unique_ids = []
it2_ingredients = []
it2_grouped = italian2_df.groupby('id')

for ids, ing in it2_grouped:
    
    it2_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    it2_ingredients.append(row)

    
mex2_unique_ids = []
mex2_ingredients = []
mex2_grouped = mexican2_df.groupby('id')

for ids, ing in mex2_grouped:
    
    mex2_unique_ids.append(ids)
    
    row = str(ing)
    row = row.replace('ingredients','')
    row = row.replace('id', '')
    row = row.replace('\n', '')
    row = re.sub('[0-9]+', '', row)
    row = row.split( )
    row = ' '.join(row)
    mex2_ingredients.append(row)


In [11]:
#using TfidfVectorizer  (italian, mexican) and adding one column Cuisine

it2_vectorizer = TfidfVectorizer(analyzer = 'word', binary = True)
it2_array = it2_vectorizer.fit_transform(it2_ingredients)
it2_array = it2_array.toarray()
it2_df = DataFrame(it2_array, columns = it2_vectorizer.get_feature_names(), index = it2_unique_ids)
it2_df['Cuisine'] = 0 #italian
it2_df.to_csv('it2_cleandata.csv', index_label = 'ID')
#print(it2_df)

mex2_vectorizer = CountVectorizer(analyzer = 'word', binary = True)
mex2_array = mex2_vectorizer.fit_transform(mex2_ingredients)
mex2_array = mex2_array.toarray()
mex2_df = DataFrame(mex2_array, columns = mex2_vectorizer.get_feature_names(), index = mex2_unique_ids)
mex2_df['Cuisine'] = 1 #mexican
mex2_df.to_csv('mex2_cleandata.csv', index_label = 'ID')
#print(mex2_df)

In [12]:
#putting mexican and italian in one DataFrame

df2 = pd.DataFrame(it2_df)
df2 = df2.append(mex2_df, sort = False)
df2.fillna(0, inplace = True)
print(df2)

       abbamele   ac  accent  acini  acorn  active  added  adobo  agave  aged  \
4           0.0  0.0     0.0    0.0    0.0     0.0    0.0    0.0    0.0   0.0   
14          0.0  0.0     0.0    0.0    0.0     0.0    0.0    0.0    0.0   0.0   
20          0.0  0.0     0.0    0.0    0.0     0.0    0.0    0.0    0.0   0.0   
56          0.0  0.0     0.0    0.0    0.0     0.0    0.0    0.0    0.0   0.0   
62          0.0  0.0     0.0    0.0    0.0     0.0    0.0    0.0    0.0   0.0   
...         ...  ...     ...    ...    ...     ...    ...    ...    ...   ...   
49678       0.0  0.0     0.0    0.0    0.0     0.0    0.0    0.0    0.0   0.0   
49687       0.0  0.0     0.0    0.0    0.0     0.0    0.0    0.0    0.0   0.0   
49707       0.0  0.0     0.0    0.0    0.0     0.0    0.0    0.0    0.0   0.0   
49709       0.0  0.0     0.0    0.0    0.0     0.0    0.0    0.0    0.0   0.0   
49717       0.0  0.0     0.0    0.0    0.0     0.0    0.0    1.0    0.0   0.0   

       ...  wensleydale  we

In [17]:
# Multiple Logistic Regression 
lr_m = linear_model.LogisticRegression()

# Support Vector Machines
svm_m = svm.LinearSVC(random_state=0)
#Decision Trees
dtree = tree.DecisionTreeClassifier()

#k-NN
knn = KNeighborsClassifier(n_neighbors = 3)

# Naive Bayes
gnb = BernoulliNB(binarize = None)

In [18]:
print('********* CountVectorizer *********')
X = df.loc[:, df.columns != 'Cuisine'].values  #all columns with values except Cuisine column
print(X.shape)

Y = df['Cuisine'] #just the ids in a list

********* CountVectorizer *********
(14276, 3976)


In [15]:
s = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
scores_ = model_selection.cross_validate(lr_m, X, Y, scoring = s, cv = 5)                                 
print('****Logistic Regression mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(lr_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Logistic Regression mean scores****
scores: {'fit_time': array([2.10236812, 2.28280258, 2.22201824, 2.11605668, 2.0456109 ]), 'score_time': array([0.0265801 , 0.0260942 , 0.02617979, 0.02655578, 0.02633691]), 'test_accuracy': array([0.96813725, 0.97197898, 0.96882662, 0.96672504, 0.96882662]), 'test_precision_weighted': array([0.96837424, 0.9720495 , 0.96895965, 0.96685339, 0.96884661]), 'test_recall_weighted': array([0.96813725, 0.97197898, 0.96882662, 0.96672504, 0.96882662]), 'test_f1_weighted': array([0.96809087, 0.97195581, 0.96879183, 0.96668815, 0.96880957])}

mean score test accuracy: 0.9688989045705847

mean score test precision weighted: 0.9690166784300608

mean score test recall weighted: 0.9688989045705847

mean score test f1-measure weighted: 0.9688672451742961
Confusion Matrix: [[7682  156]
 [ 288 6150]]


In [19]:
scores_ = model_selection.cross_validate(svm_m, X, Y, scoring = s, cv = 5)                                 
print('****SVM mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(svm_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****SVM mean scores****
scores: {'fit_time': array([0.53595734, 0.50038743, 0.50931048, 0.48424864, 0.50178266]), 'score_time': array([0.02863026, 0.02821827, 0.02669287, 0.02987385, 0.02583671]), 'test_accuracy': array([0.96148459, 0.96567426, 0.96532399, 0.96707531, 0.96497373]), 'test_precision_weighted': array([0.96156998, 0.96567343, 0.96536147, 0.9671225 , 0.96509168]), 'test_recall_weighted': array([0.96148459, 0.96567426, 0.96532399, 0.96707531, 0.96497373]), 'test_f1_weighted': array([0.96144659, 0.96566187, 0.96529947, 0.96705088, 0.96499409])}

mean score test accuracy: 0.9649063758603266

mean score test precision weighted: 0.9649638117628687

mean score test recall weighted: 0.9649063758603266

mean score test f1-measure weighted: 0.96489058154031
Confusion Matrix: [[7616  222]
 [ 279 6159]]


In [20]:
scores_ = model_selection.cross_validate(dtree, X, Y, scoring = s, cv = 5)                                 
print('****d-tree mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(dtree, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****d-tree mean scores****
scores: {'fit_time': array([15.86993647, 15.41277719, 15.96362233, 16.15348339, 10.87315965]), 'score_time': array([0.02547932, 0.02553463, 0.02614665, 0.0252068 , 0.02573919]), 'test_accuracy': array([0.94187675, 0.9502627 , 0.93625219, 0.9408056 , 0.9352014 ]), 'test_precision_weighted': array([0.94188568, 0.95026441, 0.93626193, 0.94079529, 0.9352959 ]), 'test_recall_weighted': array([0.94187675, 0.9502627 , 0.93625219, 0.9408056 , 0.9352014 ]), 'test_f1_weighted': array([0.94188075, 0.95023338, 0.9362566 , 0.9407994 , 0.93522913])}

mean score test accuracy: 0.9408797284237689

mean score test precision weighted: 0.9409006438491231

mean score test recall weighted: 0.9408797284237689

mean score test f1-measure weighted: 0.9408798521748334
Confusion Matrix: [[7411  427]
 [ 422 6016]]


In [21]:
scores_ = model_selection.cross_validate(knn, X, Y, scoring = s, cv = 5)                                 
print('****k-NN mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(knn, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****k-NN mean scores****
scores: {'fit_time': array([4.20346427, 4.18154025, 4.1372292 , 4.12692189, 4.13048267]), 'score_time': array([167.73319101, 169.05296135, 169.30660701, 167.97744846,
       168.02565145]), 'test_accuracy': array([0.88935574, 0.90262697, 0.88581436, 0.89352014, 0.89492119]), 'test_precision_weighted': array([0.89416896, 0.90782103, 0.89262856, 0.90016256, 0.89938992]), 'test_recall_weighted': array([0.88935574, 0.90262697, 0.88581436, 0.89352014, 0.89492119]), 'test_f1_weighted': array([0.88823861, 0.90164085, 0.88436686, 0.89223242, 0.89392876])}

mean score test accuracy: 0.8932476808586832

mean score test precision weighted: 0.8988342046653408

mean score test recall weighted: 0.8932476808586832

mean score test f1-measure weighted: 0.8920815014094245
Confusion Matrix: [[7539  299]
 [1225 5213]]


In [22]:
scores_ = model_selection.cross_validate(gnb, X, Y, scoring = s, cv = 5)                                 
print('****Naive-Bayes mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(gnb, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Naive-Bayes mean scores****
scores: {'fit_time': array([0.48707676, 0.47063208, 0.47015071, 0.47038531, 0.48076653]), 'score_time': array([0.08609247, 0.02903891, 0.02929163, 0.0293169 , 0.02886987]), 'test_accuracy': array([0.96148459, 0.96952715, 0.96462347, 0.96217163, 0.96812609]), 'test_precision_weighted': array([0.96237002, 0.96983328, 0.96514423, 0.96265902, 0.96816809]), 'test_recall_weighted': array([0.96148459, 0.96952715, 0.96462347, 0.96217163, 0.96812609]), 'test_f1_weighted': array([0.96137117, 0.96947715, 0.96454557, 0.96209066, 0.96810371])}

mean score test accuracy: 0.9651865860179448

mean score test precision weighted: 0.9656349263771837

mean score test recall weighted: 0.9651865860179448

mean score test f1-measure weighted: 0.9651176533708903
Confusion Matrix: [[7706  132]
 [ 365 6073]]


In [23]:
print('********* TfidfVectorizer *********')
X = df2.loc[:, df2.columns != 'Cuisine'].values  #all columns with values except Cuisine column
print(X.shape)

Y = df2['Cuisine'] #just the ids in a list

********* TfidfVectorizer *********
(14276, 2006)


In [24]:
s = ['accuracy', 'precision_weighted', 'recall_weighted', 'f1_weighted']
scores_ = model_selection.cross_validate(lr_m, X, Y, scoring = s, cv = 5)                                 
print('****Logistic Regression mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(lr_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Logistic Regression mean scores****
scores: {'fit_time': array([0.49483085, 0.48494387, 0.50219655, 0.49449825, 0.47824216]), 'score_time': array([0.00907183, 0.0092442 , 0.00911999, 0.01040626, 0.00946355]), 'test_accuracy': array([0.99264706, 0.99334501, 0.99299475, 0.99544658, 0.99404553]), 'test_precision_weighted': array([0.99274423, 0.99342468, 0.99308297, 0.99548405, 0.99410944]), 'test_recall_weighted': array([0.99264706, 0.99334501, 0.99299475, 0.99544658, 0.99404553]), 'test_f1_weighted': array([0.9926413 , 0.9933403 , 0.99298951, 0.99544444, 0.99404182])}

mean score test accuracy: 0.9936957865457916

mean score test precision weighted: 0.9937690761971514

mean score test recall weighted: 0.9936957865457916

mean score test f1-measure weighted: 0.993691474802611
Confusion Matrix: [[7838    0]
 [  90 6348]]


In [25]:
scores_ = model_selection.cross_validate(svm_m, X, Y, scoring = s, cv = 5)                                 
print('****SVM mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(svm_m, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****SVM mean scores****
scores: {'fit_time': array([0.19914865, 0.19150639, 0.19096613, 0.19335699, 0.19472957]), 'score_time': array([0.00932002, 0.0097084 , 0.00915241, 0.00927067, 0.00955343]), 'test_accuracy': array([0.99684874, 0.99544658, 0.99544658, 0.99579685, 0.99579685]), 'test_precision_weighted': array([0.99686672, 0.99547285, 0.99548403, 0.99582879, 0.99582879]), 'test_recall_weighted': array([0.99684874, 0.99544658, 0.99544658, 0.99579685, 0.99579685]), 'test_f1_weighted': array([0.99684772, 0.99544477, 0.99544443, 0.99579503, 0.99579503])}

mean score test accuracy: 0.995867120928932

mean score test precision weighted: 0.9958962354773305

mean score test recall weighted: 0.995867120928932

mean score test f1-measure weighted: 0.9958653957896434
Confusion Matrix: [[7837    1]
 [  58 6380]]


In [26]:
scores_ = model_selection.cross_validate(dtree, X, Y, scoring = s, cv = 5)                                 
print('****d-tree mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(dtree, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****d-tree mean scores****
scores: {'fit_time': array([2.89653134, 2.98665929, 2.93397117, 2.79936361, 3.13967037]), 'score_time': array([0.01443267, 0.01437497, 0.01480556, 0.01532197, 0.01470089]), 'test_accuracy': array([0.99754902, 0.99789842, 0.99789842, 0.99789842, 0.99929947]), 'test_precision_weighted': array([0.99755991, 0.99790179, 0.99790179, 0.99790179, 0.99930037]), 'test_recall_weighted': array([0.99754902, 0.99789842, 0.99789842, 0.99789842, 0.99929947]), 'test_f1_weighted': array([0.99754841, 0.99789813, 0.99789813, 0.99789813, 0.99929943])}

mean score test accuracy: 0.9981087531334776

mean score test precision weighted: 0.9981131296658168

mean score test recall weighted: 0.9981087531334776

mean score test f1-measure weighted: 0.9981084437066656
Confusion Matrix: [[7837    1]
 [  27 6411]]


In [27]:
scores_ = model_selection.cross_validate(knn, X, Y, scoring = s, cv = 5)                                 
print('****k-NN mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(knn, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****k-NN mean scores****
scores: {'fit_time': array([2.31949806, 2.30487847, 2.29804659, 2.35382581, 2.30064058]), 'score_time': array([80.37389684, 80.13014698, 80.33800316, 80.10853553, 81.24009514]), 'test_accuracy': array([0.92927171, 0.93625219, 0.93485114, 0.93625219, 0.93730298]), 'test_precision_weighted': array([0.93734352, 0.94288196, 0.94175974, 0.94288575, 0.94373068]), 'test_recall_weighted': array([0.92927171, 0.93625219, 0.93485114, 0.93625219, 0.93730298]), 'test_f1_weighted': array([0.9284027 , 0.93557536, 0.93413779, 0.93557844, 0.93665572])}

mean score test accuracy: 0.9347860405107753

mean score test precision weighted: 0.9417203315935836

mean score test recall weighted: 0.9347860405107753

mean score test f1-measure weighted: 0.934070001235187
Confusion Matrix: [[7838    0]
 [ 931 5507]]


In [28]:
scores_ = model_selection.cross_validate(gnb, X, Y, scoring = s, cv = 5)                                 
print('****Naive-Bayes mean scores****')
print('scores:', scores_)
print('\nmean score test accuracy:', scores_['test_accuracy'].mean())
print('\nmean score test precision weighted:', scores_['test_precision_weighted'].mean())
print('\nmean score test recall weighted:', scores_['test_recall_weighted'].mean())
print('\nmean score test f1-measure weighted:', scores_['test_f1_weighted'].mean())

Y_pred = cross_val_predict(gnb, X, Y, cv = 5)
conf_matrix = confusion_matrix(Y, Y_pred)
print('Confusion Matrix:', conf_matrix)

****Naive-Bayes mean scores****
scores: {'fit_time': array([0.11876631, 0.09500575, 0.09351659, 0.09325671, 0.09411526]), 'score_time': array([0.01413131, 0.0099349 , 0.01023459, 0.0100646 , 0.01016808]), 'test_accuracy': array([0.97268908, 0.97863398, 0.97513135, 0.97443082, 0.98143608]), 'test_precision_weighted': array([0.97398327, 0.97943405, 0.97620864, 0.97556896, 0.98204342]), 'test_recall_weighted': array([0.97268908, 0.97863398, 0.97513135, 0.97443082, 0.98143608]), 'test_f1_weighted': array([0.97259409, 0.97857849, 0.97505392, 0.97434897, 0.98139545])}

mean score test accuracy: 0.9764642599596757

mean score test precision weighted: 0.9774476686666732

mean score test recall weighted: 0.9764642599596757

mean score test f1-measure weighted: 0.9763941854374604
Confusion Matrix: [[7838    0]
 [ 336 6102]]
